## Imports

In [77]:
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project

In [78]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

## Contract objects

In [79]:
# Load Uniswap factory contract
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")

# Load a referece pool; obtain `contract_type` from here to read any uniswap pool with ape
pool_ref = Contract('0x8f8EF111B67C04Eb1641f5ff19EE54Cda062f163')

# USDC/ETH 0.05%
pool = Contract("0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640", contract_type=pool_ref.contract_type)

token0 = Contract(pool.token0())
token1 = Contract(pool.token1())

## Get prices

#### Inputs for querying and saving price data

In [80]:
# start_block = 12376729  # ETH/USDC 0.05% pool deployment block
start_block = chain.blocks.head.number - 10_000
head_block = chain.blocks.head.number
step = 2400
filename = f"data/price_{pool.address}_{start_block}_{head_block}_{step}.csv"

#### Helpful functions

In [81]:
def get_price_from_slot0(slot0):
    price = (slot0.sqrtPriceX96**2) * (10**(token0.decimals() - token1.decimals())) / 2**(96*2) 
    return price

#### Query prices

In [82]:
# query for tick data each block and insert into pandas dataframe
is_head = True
for block in range(start_block, head_block, step):
    print(f"Processing block {block} ...", end='\r')
    
    # get the sqrt price data at block
    slot0 = pool.slot0(block_identifier=block)
    row = {
        'block_number': [block],
        'timestamp': [chain.blocks[block].timestamp],
        'sqrt_price_x96': [slot0.sqrtPriceX96],
        'price': [get_price_from_slot0(slot0)]
    }
        
    # convert to pd dataframe then append to file
    df = pd.DataFrame(data=row)
    df.to_csv(filename, mode='a', index=False, header=is_head)
        
    if is_head:
        is_head = False

In [83]:
df = pd.read_csv(filename)
df.head()

,block_number,timestamp,sqrt_price_x96,price
0,17935724,1692289631,1903284617606487503521622641355190,0.000577
1,17938124,1692318683,1924924407329726964586907845978349,0.000590
2,17940524,1692347759,1930261184187508410041308716043573,0.000594
3,17942924,1692376835,1943473843300152630633946025932797,0.000602
4,17945324,1692405815,1938908269798720396189251708585113,0.000599
